In [1]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 109 (delta 7), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (109/109), 1.27 MiB | 5.42 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4 MB 4.2 MB/s 
     |████████████████████████████████| 448 kB 33.9 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-02-22 13:08:48--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c0:3470, 2406:da00:ff00::3403:4be7, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Loca

In [2]:
import torch.nn as nn
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
import random
import time
import datetime
import sys
from torchtext import data
from konlpy.tag import Mecab
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from torchtext.legacy import data

In [59]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import csv
import pandas as pd



data=pd.read_csv('/content/drive/MyDrive/eda_data_1m.csv', encoding='utf-8')
data.shape

(49595, 2)

In [6]:
data.sample(n=5)

,q,label
16582,롯데리아 햄버거먹다가 뼈가 나왔는데 대처를 어떻게 해야하죠??,0
14823,자꾸 조급한 맘이 드는 건 뭐지요?,0
47407,먼 나라놈들이 숫자놀음 하네~에휴,0
29553,"무조건 안된다라고 하면, 너무 옥죄이는 것 같고...",0
19597,1위거머쥐자마자 진짜 태풍올줄이야..,0


In [7]:
data['label'].value_counts()

0    38594
1    11001
Name: label, dtype: int64

In [44]:
train = data.sample(frac = 0.7, random_state = 2022)
test = data.drop(train.index)

len(train), len(test)

(34716, 14879)

## 전처리

In [15]:
import re

In [43]:
#부호를 제거해주는 함수
def alpha_num(text):
    korean = re.compile('[^ 가-힣+]')
    result = korean.sub("",text)
    return result

In [12]:
# 불용어
stop_words = ['을', '를', '이', '가', '는']

In [13]:
# 토크나이즈
tokenizer = Mecab()

In [32]:
# 토크나이징
def tokenizing(text):
  final_text = []
  words = tokenizer.morphs(text)
  for word in words:
    final_text.append(word.strip())
  return " ".join(final_text)

In [34]:
tokenizer.morphs(data['q'][1])

['배우', '려고', '하', '는', '모습', '이', '안', '보여']

In [46]:
train['q'] = train['q'].apply(alpha_num).apply(tokenizing)
test['q'] = test['q'].apply(alpha_num).apply(tokenizing)

train.head()

,q,label
17619,와 방금 궁금 해서 영상 봤 는데 심각 했 네,0
3831,어제 카톡 했 더라,1
22541,누누히 말 하 지만 사생 은 팬 이 아님 그냥 스토킹 범죄자 일 뿐,0
35067,둘 이 같이 한다니 의외 네,0
27594,하 는 짓거리 보 면 박사모 일 베이 쓰레기 들 이 진정 빨갱이 들,0


In [48]:
word_set = []
max_len = 0

for d in train['q']:
  word_set = word_set + d.split(" ")
  if len(d.split()) > max_len:
    max_len = len(d.split())
word_set = set(word_set)

In [49]:
word_to_idx = {word : i+1 for i, word in enumerate(word_set)}
print(len(word_set))
print(max_len)

21460
220


In [51]:
def word_to_key(text):
  final_text = []
  for word in text.split():
    final_text.append(word_to_idx[word])
  if len(final_text) < max_len:
    final_text = final_text +[0] *(max_len - len(final_text))
  return final_text

train['word_to_key'] = train['q'].apply(word_to_key)

train.sample(5)

,q,label,word_to_key
24890,대한민국 에서 태극기 집회 는 보도 도 안 하 고 폄하 하 는 나라 이게 나라 냐 ...,0,"[7667, 13760, 309, 14554, 15375, 11243, 15214,..."
49501,출시 발표 나 나오 지 도 않 은 걸 거품 물 고 까네 들,0,"[13978, 19665, 13635, 6152, 5734, 15214, 18800..."
47162,최순 실 이랑 이경재 노 을 감옥 에서 소리 없이 죽여 버려라 짐승 만 도 못한 쓰...,0,"[20635, 14423, 16900, 21066, 9765, 15110, 8133..."
18306,이게 쌍수 한 눈 같 아요,0,"[17521, 17841, 12905, 20576, 1301, 2020, 0, 0,..."
32786,니 가 아직 많이 생각나 네,0,"[4290, 19050, 15905, 7471, 7699, 7144, 0, 0, 0..."


## 모델 구성 및 학습

In [54]:
len(train)

34716

In [55]:
X_train = train.iloc[:25000, 2]
X_test = train.iloc[25000:, 2].reset_index(drop = True)

y_train = train.iloc[:25000, 1]
y_test = train.iloc[25000:, 1].reset_index(drop = True)

In [74]:
X_train.head()

17619    [19309, 17603, 16585, 1693, 737, 10263, 4007, ...
3831     [9396, 10118, 6881, 17037, 0, 0, 0, 0, 0, 0, 0...
22541    [5378, 19931, 20613, 14608, 12890, 623, 9507, ...
35067    [19044, 9783, 2062, 12679, 15518, 7144, 0, 0, ...
27594    [20613, 15375, 14544, 15431, 8826, 20847, 2001...
Name: word_to_key, dtype: object

In [91]:
y_test.head()

0    0
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [57]:
from torch.utils.data import Dataset, DataLoader

In [79]:
class CustomDataset(Dataset):
  def __init__(self):
    
    self.x_data = X_train
    self.y_data = [[y] for y in y_train]

  def __len__(self):

    return len(self.x_data)

  def __getitem__(self, idx):

    x = torch.LongTensor(self.x_data[idx]).to(device)
    y = torch.LongTensor(self.y_data[idx]).to(device)

    return x,y

In [80]:
class CustomDataset_test(Dataset):
  def __init__(self):
    
    self.x_data = X_test
    self.y_data = [[y] for y in y_test]

  def __len__(self):

    return len(self.x_data)

  def __getitem__(self, idx):

    x = torch.LongTensor(self.x_data[idx])
    y = torch.LongTensor(self.y_data[idx])

    return x,y

In [83]:
batch_size = 30
lr = 0.001
epochs = 30

In [84]:
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=batch_size)

In [85]:
class RNN(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
      super(RNN, self).__init__()
      self.n_layers = n_layers
      self.hidden_dim = hidden_dim

      self.embed = nn.Embedding(n_vocab, embed_dim)
      self.dropout = nn.Dropout(dropout_p)
      self.rnn = nn.RNN(embed_dim, self.hidden_dim,batch_first = True)
      self.out = nn.Sequential(
          nn.Linear(self.hidden_dim, n_classes),
          nn.Sigmoid()
      )
    def forward(self, x):
      x = self.embed(x)
      h_0 = self._init_state(batch_size=x.size(0))
      x, _ = self.rnn(x,h_0)
      h_t = x[:,-1,:]
      self.dropout(h_t)
      logit = self.out(h_t)
      return logit

    def _init_state(self, batch_size=1):
      weight = next(self.parameters()).data
      return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [86]:
n_vocab = len(word_set)+1
embedd_size = 5
hidden_size = 100
output_size = 1

In [87]:
net = RNN(1, 256, n_vocab, embedd_size, output_size, 0.2).to(device)

In [68]:
from torch import optim

In [88]:
criterion = torch.nn.BCELoss().to(device)
optimizer = optim.Adam(net.parameters(), lr)

In [ ]:
losses = []
for epoch in range(epochs):
  
  for x, y in dataloader:
    optimizer.zero_grad()
    y = y.to(device)

    # forward 연산
    hypothesis = net(x)

    # 비용 함수
    y = y.squeeze()
    cost = criterion(hypothesis, y)
    cost.backward()
    optimizer.step()
    losses.append(cost.item()) # 값만 가져오기 위해서 .item()

  # 10의 배수에 해당되는 에포크마다 비용을 출력
  if epoch % 10 == 0:
      print(epoch, cost.item())

In [92]:
dataset = CustomDataset_test()
test_loader = DataLoader(dataset, batch_size=batch_size)

In [93]:
correct = 0

with torch.no_grad():
  net = net.to('cpu')
  net.eval()
  for data, target in test_loader:
    data, target = data, target
    output = net(data)
    
    pred = output.max(1, keepdim=True)[1]
    correct += pred.eq(target.view_as(pred)).sum().item()

test_accuracy = correct / len(test_loader.dataset)
print('Accuracy:', test_accuracy)

Accuracy: 0.7756278303828736
